<a href="https://colab.research.google.com/github/Bbeelina/Trash/blob/main/Boost_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install catboost
!pip install lightgbm
!pip install xgboost


In [12]:
!pip install "dask[dataframe]"

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 7.5 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [4]:
# Генерация синтетических данных
X, y = make_regression(n_samples=1000, n_features=10, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#print(X_train, X_test, y_train, y_test)

In [7]:
# CatBoost
import catboost as cb
model_catboost = cb.CatBoostRegressor(iterations=100, verbose=0, random_seed=42) # Уменьшили итерации для скорости
model_catboost.fit(X_train, y_train)
y_pred_catboost = model_catboost.predict(X_test)
rmse_catboost = np.sqrt(mean_squared_error(y_test, y_pred_catboost))
print(y_pred_catboost)

[  17.32327916   83.58875692    5.34679408 -287.87476121   32.61345433
   38.7107511  -142.7795775   -69.40328662  -11.81440591  180.10769572
 -316.44671278  -38.4731633    30.06382836  -87.65674155  138.18122483
 -147.84813428  -51.14262696   26.45336262   89.90124876  -60.33906979
   96.12481117  197.88094792  202.41667303 -118.41320601  187.2667114
 -119.0319714  -231.47682931  216.10604455   45.7186521   233.08953919
  222.90592687 -167.89827483   20.94212921   46.12712411  -56.29947135
 -171.65916375   41.20553495  -30.69089514   18.72696619  -75.46962024
   32.38106892  -16.91226104   27.03760376  112.91390061  -54.74684422
  -20.33506243  -79.42220691 -115.13653216  118.00101566 -124.30210007
  -32.73526037  181.13294907  243.04498343  -79.72894558  -35.18146716
  -86.63640062  137.28661572   62.56001014   77.80353108  -10.07187419
 -122.2398008     5.46116145  -20.95867029  -68.34391389  141.03362462
  181.2877667   230.50111051   75.10846766  127.66522581   75.75013822
   22.2

In [15]:
# LightGBM
import lightgbm as lgb
model_lgbm = lgb.LGBMRegressor(n_estimators=100, random_state=42, force_col_wise=True) # Уменьшили количество деревьев для скорости
model_lgbm.fit(X_train, y_train)
y_pred_lgbm = model_lgbm.predict(X_test)
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))

[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 10
[LightGBM] [Info] Start training from score -0.809522
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [16]:
# XGBoost
import xgboost as xgb
model_xgboost = xgb.XGBRegressor(n_estimators=100, random_state=42) # Уменьшили количество деревьев для скорости
model_xgboost.fit(X_train, y_train)
y_pred_xgboost = model_xgboost.predict(X_test)
rmse_xgboost = np.sqrt(mean_squared_error(y_test, y_pred_xgboost))


In [17]:
# Вывод результатов
print(f"CatBoost RMSE: {rmse_catboost:.2f}")
print(f"LightGBM RMSE: {rmse_lgbm:.2f}")
print(f"XGBoost RMSE: {rmse_xgboost:.2f}")

CatBoost RMSE: 23.28
LightGBM RMSE: 31.90
XGBoost RMSE: 42.73


In [1]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 7.7 MB/s eta 0:00:00


In [7]:
import category_encoders as ce
import pandas as pd

# Make dataset
train = pd.DataFrame({
    'color': ["red", "blue", "blue", "green", "red",
              "red", "black", "black", "blue", "green"],

    'interests': ["sketching", "painting", "instruments",
                  "sketching", "painting", "video games",
                  "painting", "instruments", "sketching",
                  "sketching"],

    'height': [68, 64, 87, 45, 54, 64, 67, 98, 90, 87],

    'grade': [1, 2, 3, 2, 3, 1, 4, 4, 2, 3], })
print(train)

# Define train and target
target = train[['grade']]
train = train.drop('grade', axis = 1)

# Define catboost encoder
cbe_encoder = ce.cat_boost.CatBoostEncoder()

# Fit encoder and transform the features
cbe_encoder.fit(train, target)
train_cbe = cbe_encoder.transform(train)
print()
print('trained data')
print(train_cbe)
# We can use fit_transform() instead of fit()
# and transform() separately as follows:
# train_cbe = cbe_encoder.fit_transform(train,target)

   color    interests  height  grade
0    red    sketching      68      1
1   blue     painting      64      2
2   blue  instruments      87      3
3  green    sketching      45      2
4    red     painting      54      3
5    red  video games      64      1
6  black     painting      67      4
7  black  instruments      98      4
8   blue    sketching      90      2
9  green    sketching      87      3

trained data
   color  interests  height
0  1.875   2.100000      68
1  2.375   2.875000      64
2  2.375   3.166667      87
3  2.500   2.100000      45
4  1.875   2.875000      54
5  1.875   2.500000      64
6  3.500   2.875000      67
7  3.500   3.166667      98
8  2.375   2.100000      90
9  2.500   2.100000      87
